Importing deepstack for powerful ensembles, [more info](https://github.com/jcborges/DeepStack)

In [1]:
# !pip install deepstack

In [2]:
# import splitfolders
# splitfolders.ratio('/home/bce19229/19bce229/Dataset/CDS/Class-Divided-SET/','/home/bce19229/19bce229/Dataset/CDS/Class-Divided-SET/Output',seed=1337, ratio=(.8, 0.1,0.1))

**Importing essential pacakages**

In [3]:
# import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

2022-12-11 15:53:25.488605: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Specifying global parameters

In [4]:
img_h,img_w= (224,224)
batch_size=128
epochs=100
n_class=3

*Concatenating train and test directory paths..*

In [5]:
base_dir = '/home/bce19229/19bce229/Dataset/CDS/Class-Divided-SET/Output'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

Initializing train and test datagenerators

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
         rescale=1./255,
         rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

test_datagen= ImageDataGenerator(rescale=1./255)

In [7]:
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=3,
                                         cooldown=2,
                                         min_lr=1e-10,
                                         verbose=1)

callbacks = [reduce_learning_rate]
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

Compiling Models

In [8]:
train_generator = train_datagen.flow_from_directory(
                    train_dir,                   # This is the source directory for training images
                    target_size=(img_h, img_w),  # All images will be resized to 300x300
                    batch_size=batch_size,
                    class_mode='categorical')


from tensorflow.keras.preprocessing.image import ImageDataGenerator
validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(img_h, img_w),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 4759 images belonging to 3 classes.
Found 598 images belonging to 3 classes.


Now lets fit the models on our dataset

In [9]:
# history_1 = model_1.fit(
#       train_generator,
#       steps_per_epoch=6528//batch_size, 
#       epochs=epochs,
#       validation_data=validation_generator,
#       validation_steps=3312//batch_size,  
#       callbacks=callbacks,
#       verbose=1)

In [10]:
# history_2 = model_2.fit(
#       train_generator,
#       steps_per_epoch=6528//batch_size, 
#       epochs=epochs,
#       validation_data=validation_generator,
#       validation_steps=3312//batch_size,  
#       callbacks=callbacks,
#       verbose=1)

In [11]:
# from keras.models import load_model
# model1 = tf.keras.models.load_model('/home/bce19229/19bce229/Code/saved Models/Pretrained DenseNet121/DenseNet121-model.h5')
model2 = tf.keras.models.load_model('/home/bce19229/19bce229/Code/saved Models/Pretrained DenseNet169/DenseNet169-model.h5')
model3 = tf.keras.models.load_model('/home/bce19229/19bce229/Code/saved Models/Pretrained InceptionResNetV2/InceptionResNetV2-best-model.h5')

2022-12-11 15:53:46.978848: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-11 15:53:46.981291: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-11 15:53:47.016375: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-11 15:53:47.016427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mtechcse-Precision-7920-Tower): /proc/driver/nvidia/version does not exist
2022-12-11 15:53:47.017392: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the a

In [12]:
model4 = tf.keras.models.load_model('/home/bce19229/19bce229/Code/saved Models/Pretrained mobilenetv2/mobilenetv2-best-model.h5')

In [13]:
from deepstack.base import KerasMember
# with tf.device('/GPU:0'):
# member1 = KerasMember(name="model1", keras_model=model1, train_batches=train_generator, val_batches=validation_generator)
member2 = KerasMember(name="model2", keras_model=model2, train_batches=train_generator, val_batches=validation_generator)
member3 = KerasMember(name="model3", keras_model=model3, train_batches=train_generator, val_batches=validation_generator)
member4 = KerasMember(name="model4", keras_model=model4, train_batches=train_generator, val_batches=validation_generator)
# member5 = KerasMember(name="model5", keras_model=model5, train_batches=train_generator, val_batches=validation_generator)

/home/bce19229/miniconda3/envs/tf_gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2022-12-11 15:54:39.653135: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-11 15:54:39.671719: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


25/38 [==================>...........] - ETA: 5:40

In [ ]:
from deepstack.ensemble import DirichletEnsemble
from sklearn.metrics import accuracy_score

wAvgEnsemble = DirichletEnsemble(N=10000, metric=accuracy_score)
wAvgEnsemble.add_members([member4, member2, member3])
wAvgEnsemble.fit()
wAvgEnsemble.describe()

model1 - Weight: 0.0362 - accuracy_score: 0.7180
model2 - Weight: 0.9638 - accuracy_score: 0.7392
DirichletEnsemble accuracy_score: 0.7418


In [ ]:
from deepstack.ensemble import StackEnsemble
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

#Ensure you have the scikit-learn version >= 0.22 installed
print("sklearn version must be >= 0.22. You have:", sklearn.__version__)

stack = StackEnsemble()

# 2nd Level Meta-Learner
estimators = [
    ('rf', RandomForestClassifier(verbose=0, n_estimators=100, max_depth=15, n_jobs=20, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=100, max_depth=10, n_jobs=20, min_samples_split=20)),
    ('dtc',DecisionTreeClassifier(random_state=0, max_depth=3))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stack.model = clf
stack.add_members([member1, member2, member 3])
stack.fit()
stack.describe(metric=sklearn.metrics.accuracy_score)

sklearn version must be >= 0.22. You have: 1.0.2
Calling predict
model1 - accuracy_score: 0.7180
model2 - accuracy_score: 0.7392
StackEnsemble accuracy_score: 0.5736


/home/bce19229/miniconda3/envs/tf_gpu/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5735785953177257

Now lets save our stack-ensemble model.

Saves meta-learner and base-learner of ensemble into folder / directory.

**Args:**
* folder: the folder where models should be saved to(Create if not exists).           

In [16]:
stack.save()

<StackEnsemble: [model1, model2]>

Loading the model is as simple as this.

In [17]:
stack.load()

Loaded model1
Loaded model2


<StackEnsemble: [model1, model2]>

Don't forget to save the DCNN itself, otherwise you may have trouble.

In [ ]:
# model_json = model_1.to_json()
# with open("VGG_19.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model_1.save_weights("VGG_19_weights.h5")
# print("Saved VGG19 to disk")

NameError: name 'model_1' is not defined

In [ ]:
# model_json = model_2.to_json()
# with open("Inception_V3.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model_2.save_weights("Inception_V3_weights.h5")
# print("Saved Inception_V3 to disk")

Saved Inception_V3 to disk


**I encourage all of you to play around with this kernel, change the hyperparameters and meta learners. This ensembling is going to help you a lot in different kaggle competitions.And please give me an upvote, it motivates me to work more. :)**